In [2]:
# starting with imports
import requests
from datetime import time
import pandas as pd
import schedule 
import time
import datetime

In [3]:
# info we need to auth
CLIENT_ID = 'cEnM0dAv-0q7aC2ydC8fUQ'
SECRET_KEY = 'xejVnIGWyZoQuHFNZqaU8ckOjTVqCA'

In [4]:
# auth with pswrd file
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

with open('pw.txt','r') as f:
    pw = f.read()
    
data = {
    'grant_type':'password',
    'username':'gingerdeer632',
    'password': pw
}

In [5]:
# apply auth to future requests
headers = {'User-Agent': 'MyAPI/0.0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                   auth=auth, data=data, headers=headers)

TOKEN = res.json()['access_token']

headers['Authorization'] = f'bearer {TOKEN}'

In [6]:
# get initial request
r = requests.get('https://oauth.reddit.com/r/popular/hot',
                headers=headers)
# initialize dataframe
df_r = pd.DataFrame(columns=['id','created_time','current_time','time_since_post','subreddit','title', 'contents','upvotes','downvotes','num_comments'])

#initialize request queue
reqs = []
reqs.append(r)

In [7]:
# extraction funcs, only work in a for loop

def extract_title(result):
    try:
        return result['data']['title']
    except:
        return 'NaN'

def extract_time(result):
    try:
        ts = datetime.datetime.fromtimestamp(result['data']['created_utc'])
        return ts.strftime('%Y-%m-%d %H:%M:%S')
    except:
        return 'NaN'

def extract_subreddit(result):
    try:
        return result['data']['subreddit']
    except:
        return 'NaN'

def extract_num_comments(result):
    try:
        return result['data']['num_comments']
    except:
        return 'NaN'

def extract_id(result):
    try:
        return result['kind'] + '_' + result['data']['id']
    except:
        return 'NaN'
def extract_body(result):
    try:
        if not result['data']['selftext']:
            url = result['data']['url']
            is_photo = ['png','jpeg','jpg']
            
            if result['data']['is_video']:
                return 'Video: ' + url
            elif any(x in url for x in is_photo):
                return 'Photo: ' + url
            else:
                return 'URL: ' + result['data']['url']
        else:
            return result['data']['selftext']
    except:
        return 'NaN'
def extract_downvotes(result):
    try:
        score = result['data']['score']
        ratio = result['data']['upvote_ratio']
        return round((score*(ratio-1)) / (1-(2*ratio)))
    except:
        return 'NaN'
def extract_upvotes(result):
    try:
        downvotes = extract_downvotes(result)
        score = result['data']['score']
        return score + downvotes
    except:
        return 'NaN'
def extract_time_diff(result):
    try:
        current_dt = datetime.datetime.fromtimestamp(round(time.time()))
        post_dt = datetime.datetime.fromtimestamp(result['data']['created_utc'])
        time_d = current_dt - post_dt
        return str(time_d)
    except:
        return 'NaN'
def extract_num_awards(result):
    try:
        return result['data']['total_awards_received']
    except:
        return 'NaN'

In [8]:
# fill dataframe, change request to next page

def scrape(reqs, headers):
    r = reqs.pop(0)
   
    for post in r.json()['data']['children']:
        global df_r
        df_r = df_r.append({
            'id': extract_id(post),
            'created_time': extract_time(post),
            'current_time': datetime.datetime.fromtimestamp(round(time.time())),
            'time_since_post': extract_time_diff(post),
            'subreddit': extract_subreddit(post),
            'title': extract_title(post),
            'contents': extract_body(post),
            'upvotes': extract_upvotes(post),
            'downvotes':extract_downvotes(post),
            'num_comments': extract_num_comments(post)
        }, ignore_index=True)
            
   
    reqs.append(requests.get('https://oauth.reddit.com/r/popular/hot',
                headers=headers, params={'after':extract_id(post)}))

# save dataframe to csv occationally
def save(DataFrame):
    DataFrame.to_csv('./data/reddit_scrap.csv',mode='a',header=False, index=False)


In [9]:
# scrapes every minute
# will run until interupted

# Scheduler params for function param not changing with global variable
schedule.every(60).seconds.do(scrape,reqs=reqs,headers=headers)

while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

In [10]:
# manually save after 30 minutes
save(df_r)